In [1]:
import random
import numpy as np
import datetime
import os
import random
import pandas as pd
import numpy as np
import operator
import random
from tqdm import tqdm

try:
    import binutil  # required to import from dreamcoder modules
except ModuleNotFoundError:
    import bin.binutil  # alt import if called as module
from dreamcoder.task import Task
from dreamcoder.program import *

from dreamcoder.task import Task
from dreamcoder.dreamcoder import *
from dreamcoder.domains.minatar.primitives import basePrimitives, tmap, taction, basePrimitivesBoolOp
from dreamcoder.grammar import Grammar
from dreamcoder.utilities import testTrainSplit, eprint, numberOfCPUs
from dreamcoder.type import arrow
from dreamcoder.domains.minatar.feature_extractor import *
from dreamcoder.domains.minatar.utilities import makeTasks
from dreamcoder.dreamcoder import commandlineArguments
from dreamcoder.utilities import numberOfCPUs
import transformers
from transformers import RobertaTokenizer, T5ForConditionalGeneration, AutoTokenizer, TrainingArguments, Trainer

In [2]:
import dill
from dreamcoder.primitiveGraph import graphPrimitivesFromGrammar

path = '/home/ma/e/eberhardinger/workspaces/T5-MinAtar-experiments/asterix/50000p-500s-3si/iter-6_seqlength-6/results.pkl'
with open(path, 'rb') as handle:
    result = dill.load(handle)
    
print(result.keys())

for k, v in result.items():
    print(k, len(v['grammar'].primitives) - len(Grammar.uniform(basePrimitives('asterix')).primitives))

dict_keys([4, 5, 6])
4 12
5 15
6 17


In [3]:
g =  v['grammar']
#for p in g.primitives:
#    print(p, type(p))

In [5]:
from dreamcoder.program import Primitive, Invented
new_ps = []
i = 0
for ll, tp, p in g.productions:
    if isinstance(p, Invented):
        new_ps.append((ll, tp, InventedWithName(f"_f{i}", p.body)))
        i+=1
    else:
        new_ps.append((ll, tp, p))

In [5]:
ps = basePrimitives('asterix') #+ new_ps

In [6]:
from dreamcoder.grammar import Grammar
grammar = Grammar.uniform(ps) # Grammar.uniform(basePrimitives('breakout', ['n','l','r']))

no_input = True
t = 0
with_input = 0
while no_input or t < 100:
    p = grammar.sample(arrow(tmap, taction), minimumDepth=10, maximumDepth=20, maxAttempts=1000)
    print(p)
    if 'and' in str(p) or 'or' in str(p):
        #print(p)
        no_input=False
        with_input+=1
    t += 1
    no_input=False
with_input

(lambda (if (gt-x? (get-obj-x (get $0 1 9)) (get-obj-x (get $0 7 2))) down-action no-op-action))
(lambda (if (gt-x? (get-obj-x (get $0 5 9)) (get-obj-x (get $0 5 9))) left-action left-action))
(lambda (if (not (not (eq-obj? (get $0 5 7) gold-obj))) right-action down-action))
(lambda (if (gt-x? (get-obj-x (get $0 4 9)) (get-obj-x (get $0 2 7))) left-action right-action))
(lambda (if (eq-obj? (get $0 8 6) (get-game-obj (get $0 8 8))) down-action up-action))
(lambda (if (eq-x? (get-obj-x (get $0 2 6)) (get-obj-x (get $0 8 3))) no-op-action up-action))
(lambda (if (eq-x? (get-obj-x (get $0 1 4)) (get-obj-x (get $0 9 9))) right-action left-action))
(lambda (if (gt-x? (get-obj-x (get $0 2 7)) (get-obj-x (get $0 0 8))) down-action right-action))
(lambda (if (not (eq-x? (get-obj-x (get $0 1 1)) (get-obj-x (get $0 9 3)))) right-action no-op-action))
(lambda (if (not (eq-y? (get-obj-y (get $0 5 2)) (get-obj-y (get $0 5 3)))) right-action down-action))
(lambda (if (eq-obj? (get $0 9 8) (get-game-

3

In [ ]:
Grammar(g.logVariable, new_ps, continuationType=g.continuationType).__dict__

In [ ]:
import numpy as np
arr = np.load('/home/ma/e/eberhardinger/workspaces/gymnax-blines/notebooks/breakout/rollouts.npy', allow_pickle=True)[0]

In [ ]:
states, actions, rewards = arr

In [4]:
def get_productions(productions):
    new_prods = []
    i = 0
    productions.sort(key=lambda x: len(str(x[-1])))
    for production in productions:
        l = production[0]
        p = production[-1]
        p_str = str(p)
                         
        if '#' in p_str and '-action' in p_str:
            p_str.replace('#', f'_f{i}')
            i+=1
            new_prods.append((l, p.infer(), Program.parse(p_str)))
            new_p_str = None
            #is inventend 
            if 'left-action' in p_str:
                new_p_str = p_str.replace('left-action', 'right-action')
            elif 'right-action' in p_str:
                new_p_str = p_str.replace('right-action', 'left-action')
            elif 'up-action' in p_str:
                new_p_str = p_str.replace('up-action', 'down-action')
            elif 'down-action' in p_str:
                new_p_str = p_str.replace('down-action', 'up-action')
            if new_p_str is not None:
                new_prods.append((l, p.infer(), Program.parse(new_p_str)))
        else:
            new_prods.append((l, p.infer(), Program.parse(p_str)))
    return new_prods
#Grammar(g.logVariable, get_productions(g.productions), continuationType=g.continuationType).primitives

In [ ]:

def convert_to_task_input(arr):
    arr = arr.squeeze()
    channels = arr.shape[0]

    new_arr = np.copy(arr[0, :, :])
    for i in range(1, channels):
        new_arr[np.array(arr[i, :, :] == 1)] = i + 1
    return new_arr.astype(int)

def convert_to_task_input(arr):
    channels = arr.shape[-1]

    new_arr = np.copy(arr[:, :, 0])
    for i in range(1, channels):
        val = i + 1
        new_arr[np.array(arr[:, :, i] == 1)] = val
    return new_arr.astype(int)

def all_equal(lst):
    return not lst or lst.count(lst[0]) == len(lst)


def makeTasks(data, env_name, rand_min=10, rand_max=50, randomChunkSize=True, fixedChunkSize=None):
    assert randomChunkSize or (not randomChunkSize and fixedChunkSize)

    if randomChunkSize:
        chunkSize = random.randint(rand_min, rand_max)
    else:
        chunkSize = fixedChunkSize

    tasks = []
    examples = []
    part = 0
    states, actions, reward = data

    state_action_pairs = list(zip(states, actions))[:55]

    for i in range(len(state_action_pairs) - chunkSize):
        examples = []
        for state, action in state_action_pairs[i: i + chunkSize]:
            input_ex = (convert_to_task_input(state),)
            output_ex = int(action)
            examples.append((input_ex, output_ex))

        # we check that the chosen actions are not all the same
        # otherwise it is too easy to find a program if all actions/output examples are the same
        # this results in programs such as (lambda (lambda forward-action))
        all_chosen_actions = list(zip(*examples))[1]
        if not all_equal(all_chosen_actions):
            tasks.append(Task(f'{env_name} size {chunkSize} part {part}',
                              arrow(tmap, taction), examples))
            part += 1
            # we reset examples and add new chunkSize tasks
            if randomChunkSize:
                chunkSize = random.randint(rand_min, rand_max)
            else:
                chunkSize = fixedChunkSize

    if len(examples) > 3:
        all_chosen_actions = list(zip(*examples))[1]
        if not all_equal(all_chosen_actions):
            tasks.append(Task(f'{env_name} size {chunkSize} part {part}',
                         arrow(tmap, taction), examples))

    # random state action sequences for more tasks
    if randomChunkSize:
        chunkSize = random.randint(rand_min, rand_max)
    else:
        chunkSize = fixedChunkSize
    examples = []
    part = 0
    already_sampled = []
    while len(state_action_pairs) - len(already_sampled) > chunkSize:
        try:
            curr_sample = random.sample([x for x in state_action_pairs if str(x) not in already_sampled], chunkSize)
        except ValueError as e:
            print('makeTasks', e)
            break

        for state, action in curr_sample:
            input_ex = (convert_to_task_input(state),)
            output_ex = int(action)
            examples.append((input_ex, output_ex))

            if chunkSize > 0 and chunkSize == len(examples):
                # we check that the chosen actions are not all the same
                # otherwise it is too easy to find a program if all actions/output examples are the same
                # this results in programs such as (lambda (lambda forward-action))
                all_chosen_actions = list(zip(*examples))[1]
                if not all_equal(all_chosen_actions):
                    tasks.append(Task(f'{env_name} size {chunkSize} random {part}', arrow(
                        tmap, taction), examples))
                    part += 1
                    # we reset examples and add new chunkSize taskss
                    examples = []
                    # random state action sequences for more tasks
                    if randomChunkSize:
                        chunkSize = random.randint(rand_min, rand_max)
                    else:
                        chunkSize = fixedChunkSize

        already_sampled += [str(x) for x in curr_sample]

    print(f'Created {len(tasks)} tasks with {fixedChunkSize} chunk size')
    return tasks


In [ ]:
makeTasks(arr, 'asterix', fixedChunkSize=50)

In [ ]:
p = Program.parse('(lambda (if (eq-obj? (get $0 9 2) (get-game-obj (get $0 8 7))) (if (eq-obj? (get $0 8 2) enemy-obj) down-action (if (eq-obj? (get $0 7 9) enemy-obj) down-action right-action)) up-action))')

In [ ]:
from dreamcoder.domains.minatar.feature_extractor import MinatarFeatureExtractorToken
feature_extractor = MinatarFeatureExtractorToken(env_name='asterix', max_steps=500, compress=False, no_spaces=True)

t = feature_extractor.taskOfProgram(p, None)
t.examples

In [ ]:
import os
import numpy as np
import dill

try:
    import binutil  # required to import from dreamcoder modules
except ModuleNotFoundError:
    import bin.binutil  # alt import if called as module


from dreamcoder.domains.minatar.primitives import basePrimitives
from dreamcoder.grammar import Grammar
from dreamcoder.domains.minatar.utils_text_encoder import Collator, ec_iterator_T5
from dreamcoder.domains.minatar.utilities import get_latest_checkpoint_path, makeTasks, createTestDataFromTasks, check_test_tasks
from dreamcoder.domains.minatar.feature_extractor import MinatarFeatureExtractorToken
from transformers import RobertaTokenizer, T5ForConditionalGeneration, TrainingArguments, ViTConfig, GPT2Config, VisionEncoderDecoderModel, VisionEncoderDecoderConfig

env_name = 'Breakout-MinAtar'
verbose = True
only_eval = False
resume = False
lib_learning = True
run_name = 'test'
output_dir = f'/home/ma/e/eberhardinger/workspaces/T5-experimens/{run_name}/'
os.makedirs(output_dir, exist_ok=True)

data = np.load(
    f"/home/ma/e/eberhardinger/workspaces/gymnax-blines/notebooks/Breakout-MinAtar/Breakout-MinAtar_1-rollouts.npy", allow_pickle=True)[0]

# we need to call this one time, as otherwise we get parsing errors
grammar = Grammar.uniform(basePrimitives(env_name))
if resume or only_eval:
    checkpoint_dir = get_latest_checkpoint_path(output_dir)
    model = T5ForConditionalGeneration.from_pretrained(checkpoint_dir).to('cuda')
    tokenizer = RobertaTokenizer.from_pretrained(checkpoint_dir)
    grammar_file = os.path.join(output_dir, 'results.pkl')
    with open(grammar_file, 'rb') as handle:
        result = dill.load(handle)
    grammar = [g['grammar'] for g in result.values()][-1]
    key = list(result.keys())[-1]
    print(f'loaded checkpoint from {checkpoint_dir} and grammar from {grammar_file} after sequence length of {key}')
else:
    model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small').to('cuda')
    tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
    print('created new model and use base primitives for the grammar')
print(f'Grammar # primitives: {len(grammar.primitives)}')

collator = Collator(tokenizer)
if only_eval:
    sequence_lengths = args.sequence_lengths
    for i in sequence_lengths:
        tasks = makeTasks(data, randomChunkSize=False, fixedChunkSize=i)
        testTasks = createTestDataFromTasks(tasks, True)
        stats = check_test_tasks(testTasks, grammar, n_sampling=5, verbose=verbose)
        solved = [x for x in stats if x[0] is not None]
        rate = len(solved)/len(stats) * 100
        print(f'{len(solved)}/{len(stats)} -> {rate}%')
else:
    training_args = TrainingArguments(per_device_train_batch_size=20, 
                                      gradient_accumulation_steps=8,
                                      save_steps = 500,
                                      num_train_epochs=3,
                                      report_to=None,
                                      output_dir = './test-vit-model/')
    ec_iterator_T5(env_name, model, tokenizer, collator, MinatarFeatureExtractorToken, grammar, data, training_args, random_programs=40, lib_learning=lib_learning, compress=True, no_spaces=True,
                   output_dir=output_dir, min_len_random_programs=20, max_len_random_programs=20, verbose=verbose)
   

In [ ]:
import os
import numpy as np
import dill

try:
    import binutil  # required to import from dreamcoder modules
except ModuleNotFoundError:
    import bin.binutil  # alt import if called as module


from dreamcoder.domains.minatar.primitives import basePrimitives
from dreamcoder.grammar import Grammar
from dreamcoder.domains.minatar.utils_vision_encoder import VisionCollator, ec_iterator_T5
from dreamcoder.domains.minatar.utilities import get_latest_checkpoint_path, makeTasks, createTestDataFromTasks, check_test_tasks
from dreamcoder.domains.minatar.feature_extractor import MinatarFeatureExtractorToken
from transformers import RobertaTokenizer, T5ForConditionalGeneration, TrainingArguments, ViTConfig, GPT2Config, VisionEncoderDecoderModel, VisionEncoderDecoderConfig

env_name = 'Breakout-MinAtar'
verbose = True
only_eval = False
resume = False
lib_learning = True
run_name = 'test'
output_dir = f'/home/ma/e/eberhardinger/workspaces/T5-experimens/{run_name}/'
os.makedirs(output_dir, exist_ok=True)

data = np.load(
    f"/home/ma/e/eberhardinger/workspaces/gymnax-blines/notebooks/Breakout-MinAtar/Breakout-MinAtar_1-rollouts.npy", allow_pickle=True)[0]

# we need to call this one time, as otherwise we get parsing errors
grammar = Grammar.uniform(basePrimitives(env_name))
if resume or only_eval:
    checkpoint_dir = get_latest_checkpoint_path(output_dir)
    model = T5ForConditionalGeneration.from_pretrained(checkpoint_dir).to('cuda')
    tokenizer = RobertaTokenizer.from_pretrained(checkpoint_dir)
    grammar_file = os.path.join(output_dir, 'results.pkl')
    with open(grammar_file, 'rb') as handle:
        result = dill.load(handle)
    grammar = [g['grammar'] for g in result.values()][-1]
    key = list(result.keys())[-1]
    print(f'loaded checkpoint from {checkpoint_dir} and grammar from {grammar_file} after sequence length of {key}')
else:
    tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
    config_encoder = ViTConfig(num_channels=10, encoder_stride=1, image_size=(60,30), patch_size=2)
    config_decoder = GPT2Config(vocab_size=len(tokenizer))
    config_decoder.is_decoder = True
    config_decoder.add_cross_attention = True
    config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
    #print(config)
    model = VisionEncoderDecoderModel(config=config)
    model.config.decoder_start_token_id = tokenizer.cls_token_id
    model.config.pad_token_id = tokenizer.pad_token_id
    print('created new model and use base primitives for the grammar')
print(f'Grammar # primitives: {len(grammar.primitives)}')

collator = VisionCollator(tokenizer)
if only_eval:
    sequence_lengths = args.sequence_lengths
    for i in sequence_lengths:
        tasks = makeTasks(data, randomChunkSize=False, fixedChunkSize=i)
        testTasks = createTestDataFromTasks(tasks, True)
        stats = check_test_tasks(testTasks, grammar, n_sampling=5, verbose=verbose)
        solved = [x for x in stats if x[0] is not None]
        rate = len(solved)/len(stats) * 100
        print(f'{len(solved)}/{len(stats)} -> {rate}%')
else:
    training_args = TrainingArguments(per_device_train_batch_size=20, 
                                      gradient_accumulation_steps=8,
                                      save_steps = 500,
                                      num_train_epochs=3,
                                      report_to=None,
                                      output_dir = './test-vit-model/')
    #ec_iterator_T5(env_name, model, collator, MinatarFeatureExtractorToken, grammar, data, training_args, random_programs=500, lib_learning=lib_learning,
      #             output_dir=output_dir, min_len_random_programs=10, max_len_random_programs=10, verbose=verbose)


In [ ]:
np.load('/home/ma/e/eberhardinger/workspaces/T5-experimens/test/iter_5-ec_iterator_T5-gen_data_50.npy', allow_pickle=True)

In [ ]:
import torch
tasks = []
for j in range(3, 6):
    tasks += makeTasks(data, env_name, randomChunkSize=False, fixedChunkSize=j)


In [ ]:
input_tensor.shape

In [ ]:
solved_programs = {
    'Breakout-MinAtar size 3 part 65': '(lambda (if (eq-obj? (get $0 0 7) (get $0 9 4)) right-action no-op-action))',
    'Breakout-MinAtar size 3 part 74': '(lambda (if (eq-obj? (get $0 3 9) trail-obj) no-op-action right-action))',
    'Breakout-MinAtar size 3 random 10': '(lambda (if (eq-obj? (get $0 1 5) brick-obj) no-op-action right-action))',
    'Breakout-MinAtar size 5 part 48': '(lambda (if (gt? 9 5) (if (eq-obj? (get $0 4 9) (get $0 2 1)) no-op-action left-action) left-action))' 
} 

test_consol = [

    "(lambda (if (gt-y? (get-obj-y (get $0 2 5)) (get-obj-y (get $0 9 1))) left-action down-action))",
    "(lambda (if (eq-obj? (get $0 1 7) trail-obj) left-action left-action))",
    "(lambda (if (gt-x? (get-obj-x (if (eq-obj? (get $0 8 5) player-obj) (get $0 5 2) (get $0 1 7))) (get-obj-x (get $0 4 9))) up-action left-action))",
    "(lambda (if (eq-obj? (get $0 9 2) (get-game-obj (get $0 1 4))) left-action no-op-action))",
]

In [ ]:
from dreamcoder.frontier import *

frontiers = []
for i, program in enumerate(test_consol):
    frontier = Frontier([FrontierEntry(program=program,
                            logLikelihood=0., logPrior=0.1)],
                            task=Task(f"Helmhotz Dream {i}", arrow(tmap, taction), []))
    frontiers.append(frontier)

In [ ]:
frontiers

In [ ]:
from dreamcoder.dreamcoder import *
result = ECResult(parameters={},
                  grammars=[grammar],
                  taskSolutions={
    f.task: f for f in frontiers},
    recognitionModel=None, numTestingTasks=10,
    allFrontiers={
    f.task: f for f in frontiers})
grammar = consolidate(result, grammar, iteration=0, arity=3, aic=1.0, pseudoCounts=30.0,
                      structurePenalty=1.5, compressor='ocaml', topK=1, CPUs=numberOfCPUs())

In [ ]:
from stitch_core import compress
from stitch_core import from_dreamcoder


